In [ ]:
import pymongo
from pprint import pprint

mongo_client = pymongo.MongoClient("mongodb://localhost:27018/")
mongo_db = mongo_client["depictioDB"]

workflow_name = "ashleys-qc-pipeline"
collection = mongo_db[workflow_name]
data_sources = collection.distinct(
    "metadata.report_general_stats_headers", {"wf_name": workflow_name}
)
pprint(data_sources)

In [ ]:
import pandas as pd

df = pd.DataFrame.from_records([d[key] | {"key": key} for d in data_sources for key in d])
df.groupby("namespace")["description"].unique().to_dict()

In [ ]:
df.groupby("namespace")["description"].unique().to_dict()

In [ ]:
dev_col = mongo_db["dev"]
dev_col.insert_many(df.to_dict("records"))

In [ ]:
# Query data from MongoDB and convert it to Arrow table
cursor = dev_col.find({})
df = pd.DataFrame(list(cursor))

# Convert Arrow table back to pandas DataFrame
df